In [1]:
import pickle
import pandas as pd
import numpy as np
# from pubchempy import get_compounds
from tqdm import tqdm
import sys
from time import sleep
import os

In [2]:
from run import repurpose

3.7.7 (default, Mar 23 2020, 22:36:06) 
[GCC 7.3.0]


In [44]:
sid, smile = zip(*pd.read_csv("final/chem.csv").to_numpy().tolist())
google, gseq = zip(*pd.read_csv("final/g_seq.csv").to_numpy().tolist())
rcsb, rseq = zip(*pd.read_csv("final/r_seq.csv").to_numpy().tolist())

for target_name in np.unique(google):
    repurpose(gseq, target_name, smile, sid, save_dir='./final/google_'+str(target_name))

Target Name:  seq-0
Loading customized repurposing dataset...
Beginning Downloading Pretrained Model...
Note: if you have already download the pretrained model before, please stop the program and set the input parameter 'pretrained_dir' to the path
Dataset already downloaded in the local system...
Using pretrained model and making predictions...
repurposing...
Drug Target Interaction Prediction Mode...
in total: 13741 drug-target pairs
encoding drug...
unique drugs: 13741
rdkit not found this smiles: [Fe] convert to all 0 features
rdkit not found this smiles: N convert to all 0 features
rdkit not found this smiles: [13NH3] convert to all 0 features
rdkit not found this smiles: [81Kr] convert to all 0 features
rdkit not found this smiles: [S] convert to all 0 features
rdkit not found this smiles: Cl convert to all 0 features
rdkit not found this smiles: [He] convert to all 0 features
rdkit not found this smiles: [99Mo] convert to all 0 features
rdkit not found this smiles: [Xe] convert 

In [45]:

google, gseq = zip(*pd.read_csv("final/g_seq.csv").to_numpy().tolist())
google = pd.read_csv("final/google_sequence.csv").drop_duplicates()

In [46]:

mapping = {}

seq_ids = []
for seq in google['seq']:
    if mapping.get(seq) is None:
        mapping[seq] = len(mapping)
    seq_ids.append(mapping[seq])
google['seq_id'] = [f"seq-{i}" for i in seq_ids]
google = google[['pdb', 'seq_id', 'seq']]

inv_map = {v: k for k, v in mapping.items()}

In [47]:
full_scores = []

for target_name in google['seq_id'].drop_duplicates():
    print(target_name)
    
    seq_folder = 'google_' + str(target_name)
    seq_file_name = f"final/{seq_folder}/results_aggregation/output_list.pkl"
    
    if os.path.exists(seq_file_name):
        out = []
        with open(seq_file_name, 'rb') as f:
            out.append(pickle.load(f))
        out = pd.DataFrame(np.array(out).reshape((-1, 3)),
                           columns=['drug', 'prot', 'score'])
        out.sort_values(by=['score'], inplace=True)
        out['score'] = out['score'].astype(np.float)
        full_scores.append(out)
    else:
        print("Skipping...")

seq-0
seq-1
seq-2
seq-3
seq-4
seq-5
seq-6
seq-7
seq-8
seq-9
seq-10
seq-11
seq-12
seq-13
seq-14
seq-15
seq-16
seq-17
seq-18
seq-19
seq-20
seq-21
seq-22
seq-23
seq-24
seq-25
seq-26
seq-27
seq-28
seq-29
seq-30
seq-31
seq-32
seq-33
seq-34
seq-35
seq-36
seq-37
seq-38
seq-39
seq-40
seq-41
seq-42
seq-43
seq-44
seq-45
seq-46
seq-47
seq-48
seq-49
seq-50
seq-51
seq-52
seq-53
seq-54
seq-55
seq-56
seq-57
seq-58
seq-59
seq-60


In [48]:
final_list = pd.concat(full_scores)
final_list.sort_values(by=['score'], inplace=True)


In [49]:
final_list['prot_seq'] = final_list['prot'].str.lstrip("seq-").astype(int)

In [50]:
final_list.replace({"prot_seq": inv_map}, inplace=True)

In [51]:
final_list

,drug,prot,score,prot_seq
0,smile-43,seq-0,0.38,YPIINFTTAGATVQSYTNFIRAVRGRLTTGADVRHEIPVLPNRVGL...
0,smile-43,seq-49,0.38,WLASLKQTLGLLPADRKIRVLMLGLDNAGKTSILYRLHLGDVVTTV...
0,smile-43,seq-16,0.38,MSRAYDLVVLGAGSGGLEAGWNAAVTHKKKVAVVDVQATHGPPLFA...
0,smile-43,seq-17,0.38,TATVREAAPQFSGQAVVNGAIKDINMNDYKGKYIVLFFYPMDFTFV...
0,smile-43,seq-48,0.38,KSLSAAAQACVKKMRDAKVNEACIRTFIAQHVMVSKGETGSIPDSA...
...,...,...,...,...
13740,smile-10201,seq-10,4836048.87,RARNVRSHTGEYAPDILVVGSCFLDYVGYVDHMPQVGETMHSVSFH...
13740,smile-10201,seq-5,4836048.87,ATVRDPAPQFSGKAVVDGAIKEINSNDYKGKYIVLFFYPMDFTFVC...
13740,smile-10201,seq-2,4836048.87,QYPIINFTTAGATVQSYTNFIRAVRGRLTTGADVRHEIPVLPNRVG...
13740,smile-10201,seq-50,4836048.87,MAANCIFCKIIKGDIPCAKVAETSKALAFMDINPLSRGHMLVIPKE...


In [52]:
final_list['drug_smiles'] = final_list['drug'].str.lstrip("smile-").astype(int)

In [53]:
drug_map = pd.read_csv("Data/cid_drug_smile.csv")

In [54]:
drug_map_dict = drug_map[['DP_index', 'smiles']].to_dict()

In [55]:
drug_map_dict = {}
for r in drug_map.iterrows():
    idx = r[1]["DP_index"]
    smiles =  r[1]["smiles"]
    if not idx in drug_map_dict:
        drug_map_dict[idx] = smiles

In [56]:
final_list.replace({"drug_smiles": drug_map_dict}, inplace=True)

In [57]:
final_list[['prot_seq','drug_smiles','score']].to_csv('final_dp.csv', index=False)

In [58]:
final_list

,drug,prot,score,prot_seq,drug_smiles
0,smile-43,seq-0,0.38,YPIINFTTAGATVQSYTNFIRAVRGRLTTGADVRHEIPVLPNRVGL...,CO[C@H]1C=CC2=CCN3CCc4cc5c(cc4[C@@]23C1)OCO5
0,smile-43,seq-49,0.38,WLASLKQTLGLLPADRKIRVLMLGLDNAGKTSILYRLHLGDVVTTV...,CO[C@H]1C=CC2=CCN3CCc4cc5c(cc4[C@@]23C1)OCO5
0,smile-43,seq-16,0.38,MSRAYDLVVLGAGSGGLEAGWNAAVTHKKKVAVVDVQATHGPPLFA...,CO[C@H]1C=CC2=CCN3CCc4cc5c(cc4[C@@]23C1)OCO5
0,smile-43,seq-17,0.38,TATVREAAPQFSGQAVVNGAIKDINMNDYKGKYIVLFFYPMDFTFV...,CO[C@H]1C=CC2=CCN3CCc4cc5c(cc4[C@@]23C1)OCO5
0,smile-43,seq-48,0.38,KSLSAAAQACVKKMRDAKVNEACIRTFIAQHVMVSKGETGSIPDSA...,CO[C@H]1C=CC2=CCN3CCc4cc5c(cc4[C@@]23C1)OCO5
...,...,...,...,...,...
13740,smile-10201,seq-10,4836048.87,RARNVRSHTGEYAPDILVVGSCFLDYVGYVDHMPQVGETMHSVSFH...,O=C(O)[C@@H]1O[C@H](O[C@H]2[C@@H](O)[C@@H](O)[...
13740,smile-10201,seq-5,4836048.87,ATVRDPAPQFSGKAVVDGAIKEINSNDYKGKYIVLFFYPMDFTFVC...,O=C(O)[C@@H]1O[C@H](O[C@H]2[C@@H](O)[C@@H](O)[...
13740,smile-10201,seq-2,4836048.87,QYPIINFTTAGATVQSYTNFIRAVRGRLTTGADVRHEIPVLPNRVG...,O=C(O)[C@@H]1O[C@H](O[C@H]2[C@@H](O)[C@@H](O)[...
13740,smile-10201,seq-50,4836048.87,MAANCIFCKIIKGDIPCAKVAETSKALAFMDINPLSRGHMLVIPKE...,O=C(O)[C@@H]1O[C@H](O[C@H]2[C@@H](O)[C@@H](O)[...
